# PyBullet demo

This notebook demonstrates how to train a USA model on a PyBullet environment. The environment is taken from [here](https://github.com/adubredu/pybullet_kitchen).

To get started, first make sure to install some dependencies:

```bash
pip install pybullet scikit-learn scipy
```

In [ ]:
import os

# These environment variables control where training and eval logs are written.
# You can set these in your shell profile as well.
os.environ["RUN_DIR"] = "runs"
os.environ["EVAL_RUN_DIR"] = "eval_runs"
os.environ["MODEL_DIR"] = "models"
os.environ["DATA_DIR"] = "data"

# This is used to set a constant Tensorboard port.
os.environ["TENSORBOARD_PORT"] = str(8989)

import ml.api as ml  # Source: https://github.com/codekansas/ml-starter

ml.configure_logging()

# Imports these files to add them to the model and task registry.
from usa.models.point2emb import Point2EmbModel
from usa.tasks.clip_sdf import ClipSdfTask

In [ ]:
import pickle as pkl
import zipfile
from pathlib import Path
from typing import Iterator

import cv2
import imageio
import matplotlib.pyplot as plt
import ml.api as ml
import numpy as np
import pybullet as pb
import requests
from IPython.display import Image
from omegaconf import OmegaConf
from pyquaternion import Quaternion
from torch import Tensor

In [ ]:
pb.connect(pb.DIRECT)
pb.resetSimulation()

pb.setGravity(0, 0, -9.81)
pb.setPhysicsEngineParameter(enableConeFriction=0)

The code below downloads the environment data and adds it to PyBullet.

In [ ]:
data_root = Path("data")
data_root.mkdir(exist_ok=True)

# Downloads the dataset, if it is not already downloaded.
if not (data_root / "04_pybullet_data").exists():
    r = requests.get("https://github.com/codekansas/usa/releases/download/v0.0.2/04_pybullet_data.zip", allow_redirects=True)
    with open(data_root / "04_pybullet_data.zip", "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile(data_root / "04_pybullet_data.zip", "r") as zip_ref:
        zip_ref.extractall(data_root)

# Loads the URDFs into PyBullet.
pb.setAdditionalSearchPath(str(data_root / "04_pybullet_data"))
kitchen_path = "kitchen_part_right_gen_convex.urdf"
use_fixed_base = True
pb.setGravity(0, 0, -9.81)

floor = pb.loadURDF(
    "floor.urdf",
    useFixedBase=use_fixed_base,
)

kitchen = pb.loadURDF(
    "kitchen_part_right_gen_convex.urdf",
    (-5, 0, 1.477),
    useFixedBase=use_fixed_base,
)

table = pb.loadURDF(
    "table.urdf",
    (1.0, 0, 0),
    pb.getQuaternionFromEuler((0, 0, 1.57)),
    useFixedBase=use_fixed_base,
)

Next, we create a new Kitchen environment.

In [ ]:
class Kitchen:
    def __init__(self):
        kitchen_path = "kitchen_part_right_gen_convex.urdf"
        useFixedBase = True
        pb.setGravity(0, 0, -9.81)
        self.floor = pb.loadURDF("floor.urdf", useFixedBase=useFixedBase)
        self.kitchen = pb.loadURDF(kitchen_path, [-5, 0, 1.477], useFixedBase=useFixedBase)
        self.table = pb.loadURDF(
            "table.urdf",
            [1.0, 0, 0],
            pb.getQuaternionFromEuler([0, 0, 1.57]),
            useFixedBase=useFixedBase,
        )

Next, let's collect some samples from the environment. The `PosedRGBDItem` used for training the model has the following description:

```python
class PosedRGBDItem(NamedTuple):
    image: Tensor       # RGB image, with shape (C, H, W)
    depth: Tensor       # Depth image, with shape (1, H, W)
    mask: Tensor        # Valid depth points, with shape (1, H, W), where True means valid
    intrinsics: Tensor  # Camera intrinsics matrix, with shape (3, 3)
    pose: Tensor        # Camera pose matrix, with shape (4, 4)

    def check(self) -> None:
        # Image should have shape (C, H, W)
        assert self.image.dim() == 3
        assert self.image.dtype == torch.float32
        # Depth should have shape (1, H, W)
        assert self.depth.dim() == 3
        assert self.depth.shape[0] == 1
        assert self.depth.dtype == torch.float32
        # Depth shape should match image shape.
        assert self.depth.shape[1:] == self.image.shape[1:]
        assert self.mask.shape[1:] == self.image.shape[1:]
        # Intrinsics should have shape (3, 3)
        assert self.intrinsics.shape == (3, 3)
        assert self.intrinsics.dtype == torch.float64
        # Pose should have shape (4, 4)
        assert self.pose.shape == (4, 4)
        assert self.pose.dtype == torch.float64
```

The intrinsics matrix can be constructed as follows:

```python
def intrinsics_matrix(fx: float, fy: float, cx: float, cy: float) -> np.ndarray:
    intr = np.eye(3, dtype=np.float64)
    intr[0, 0] = fx
    intr[1, 1] = fy
    intr[0, 2] = cx
    intr[1, 2] = cy
    return intr
```

The pose matrix can be constructed from the rotation quaternion and translation vector using this snippet:

```python
import numpy as np
from quaternion import as_rotation_matrix, quaternion  # pip install numpy-quaternion


def as_pose_matrix(pose: list[float]) -> np.ndarray:
    """Converts a list of pose parameters to a pose matrix.

    Args:
        pose: The list of pose parameters, (qx, qy, qz, qw, px, py, pz)

    Returns:
        A (4, 4) pose matrix
    """

    mat = np.eye(4, dtype=np.float64)
    qx, qy, qz, qw, px, py, pz = pose
    mat[:3, :3] = as_rotation_matrix(quaternion(qw, qx, qy, qz))
    mat[:3, 3] = [px, py, pz]
    return mat
```

In the example below, we move the camera around the center point to collect a few frames.

In [ ]:
def capture_frame(
    camera_xyz: tuple[float, float, float] = (-5.0, 0.0, 1.477),
    camera_ypr: tuple[float, float, float] = (90.0, -10.0, 0.0),
    camera_distance: float = 3.0,
    camera_planes: tuple[float, float] = (0.01, 10.0),
    pixel_dims: tuple[int, int] = (500, 300),
    camera_fov: float = 80.0,
) -> tuple[np.ndarray, ...]:
    """Captures a single frame, returning RGB and depth information.

    Args:
        camera_xyz: The XYZ coordinates of the camera
        camera_ypr: The yaw, pitch and roll of the camera
        cam_distance: Not sure
        camera_planes: The minimum and maximum rendering distances
        pixel_dims: The shape of the output image, as (W, H)
        camera_fov: The camera field of view
        
    Returns:
        The RGB image with shape (H, W, 3), the depth image with shape (H, W),
        the intrinsics matrix with shape (3, 3), and the pose matrix with
        shape (4, 4).
    """

    yaw, pitch, roll = camera_ypr
    near_plane, far_plane = camera_planes
    pixel_width, pixel_height = pixel_dims

    # Computes the view and projection matrices.
    view_mat = pb.computeViewMatrixFromYawPitchRoll(camera_xyz, camera_distance, yaw, pitch, roll, 2)
    aspect = pixel_width / pixel_height
    proj_mat = pb.computeProjectionMatrixFOV(camera_fov, aspect, near_plane, far_plane)

    # Captures the camera image.
    img_arr = pb.getCameraImage(
        width=pixel_width,
        height=pixel_height,
        viewMatrix=view_mat,
        projectionMatrix=proj_mat,
    )
    img_width, img_height, rgb, depth, info = img_arr

    # Reshapes arrays to expected output shape.
    rgb_arr = np.reshape(rgb, (img_height, img_width, 4))[..., :3]
    depth_arr = np.reshape(depth, (img_height, img_width))
    
    # Converts depth to true depth.
    depth_arr = far_plane * near_plane / (far_plane - (far_plane - near_plane) * depth_arr)
    
    # Gets camera intrinsics matrix.
    cx = pixel_width / 2
    cy = pixel_height / 2
    fov_rad = np.deg2rad(camera_fov)
    fx = cx / np.tan(fov_rad / 2)
    fy = cy / np.tan(fov_rad / 2)
    intr = np.eye(3)
    intr[0, 0] = fx
    intr[1, 1] = fy
    intr[0, 2] = cx
    intr[1, 2] = cy
    
    # Gets the pose matrix. This requires applying some
    # magic affine transformations to convert to the coordinate
    # frame expected by the framework.
    pose = np.linalg.inv(np.array(view_mat).reshape(4, 4, order="C").T)
    
    affine_mat = np.array(
        [
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, -1, 0],
            [0, 0, 0, 1],
        ]
    )
    pose = pose @ affine_mat
    
    affine_mat = np.array(
        [
            [1, 0, 0, 0],
            [0, 0, -1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1],
        ]
    )
    pose = affine_mat @ pose
    
    return rgb_arr, depth_arr, intr, pose


def move_camera(
    camera_xyz: tuple[float, float, float],
    camera_ypr: tuple[float, float, float],
) -> tuple[tuple[float, float, float], tuple[float, float, float]]:
    x, y, z = camera_xyz
    yaw, pitch, roll = camera_ypr
    return (
        (x - 0.005, y + 0.01, z),
        (yaw + 0.1, pitch, roll),
    )


def capture_sim(capture_every: int = 1) -> Iterator[tuple[np.ndarray, ...]]:
    # xyz, ypr = (-5.0, 0.0, 1.477), (90.0, -10.0, 0.0)
    xyzs, yprs = [(-5.0, 0.0, 1.477)] * 90, [(i * 4, -10.0, 0.0) for i in range(90)]
    for i, (xyz, ypr) in enumerate(zip(xyzs, yprs)):
        if i % capture_every == 0:
            yield capture_frame(xyz, ypr)


def write_gif(frames: Iterator[tuple[np.ndarray, ...]], out_file: str | Path, pkl_file: str | Path, *, fps: int = 30) -> None:
    rgb, depth, mask, poses, intrinsics = [], [], [], [], []

    writer = imageio.get_writer(str(out_file), mode="I", fps=fps)
    for rgb_frame, depth_frame, intr, pose in frames:
        # Adds to the lists.
        rgb.append(rgb_frame)
        depth.append(depth_frame)
        mask.append(depth_frame > 7.0)
        # mask.append(np.zeros_like(depth_frame, dtype=np.bool_))
        poses.append(pose)
        intrinsics.append(intr)

        # Adds the image to the GIF.
        depth_normalized = (depth_frame - np.min(depth_frame)) / (np.max(depth_frame) - np.min(depth_frame) + 1e-3)
        depth_colorized = (plt.cm.jet(depth_normalized)[..., :3] * 255).astype(np.uint8)
        frame = np.concatenate([rgb_frame, depth_colorized], axis=0)
        writer.append_data(frame)

    # Saves the pickle file.
    data = {
        "rgb": np.stack(rgb),
        "depth": np.stack(depth),
        "mask": np.stack(mask),
        "poses": np.stack(poses),
        "intrinsics": np.stack(intrinsics),
    }
    with open(pkl_file, "wb") as f:
        pkl.dump(data, f)

    # Closes the writer.
    writer.close()


def iter_frames() -> Iterator[np.ndarray]:
    pb.resetSimulation()
    kitchen = Kitchen()
    yield from capture_sim()


pkl_path = data_root / "04_recorded_clip.pkl"
write_gif(iter_frames(), "video.gif", pkl_path)
Image("video.gif")

We can visualize the point cloud for the dataset using the snippet below. Note that this requires `pythreejs`, which can be installed using:

```bash
pip install pythreejs
```

Also, if using a normal Jupyter notebook, you will need to enable the extension using the code below (see the project page [here](https://github.com/jupyter-widgets/pythreejs)):

```bash
jupyter nbextension list
jupyter nbextension install --py --symlink --sys-prefix pythreejs
jupyter nbextension enable --py --sys-prefix pythreejs
jupyter nbextension list
```

In [ ]:
if False:  # This is just for testing
    from usa.tasks.datasets.utils import visualize_posed_rgbd_dataset
    from usa.tasks.datasets.pybullet import PyBulletDataset

    # Creates a new dataset from the recorded pickle file.
    dataset = PyBulletDataset(path=pkl_path)

    # Creates a point cloud of the dataset.
    out_path = Path("out/point_cloud.ply")
    visualize_posed_rgbd_dataset(
        dataset,
        make_video=False,
        make_point_cloud=True,
        max_point_cloud_samples=3,
        point_cloud_sample_stride=5,
        output_dir=out_path.parent,
    )

    # Plots the point cloud using PyVista.
    import pyvista as pv
    pv.set_jupyter_backend("trame")
    cloud = pv.read("out/point_cloud.ply")
    cloud.plot()

Next, we train a model on the recorded clip.

In [ ]:
# Using the default config, but overriding the dataset.
config = OmegaConf.load("config.yaml")
config.task.dataset = "pybullet"
config.task.dataset_path = str(pkl_path)

# We still need to explicitly set these variables.
config.trainer.exp_name = "jupyter"
config.trainer.base_run_dir = "runs"
config.trainer.run_id = 0

# Only use stdout logger.
config.logger = [{"name": "stdout"}]

# You can change this number to change the number of training steps.
config.task.finished.max_steps = 500

# Loads the config objects.
objs = ml.instantiate_config(config)

# Unpacking the different components.
model = objs.model
task = objs.task
optimizer = objs.optimizer
lr_scheduler = objs.lr_scheduler
trainer = objs.trainer

# Runs the training loop.
trainer.train(model, task, optimizer, lr_scheduler)

In [ ]:
# If a model has already been trained, you can load trained model and task
# using this line:
# model, task = ml.load_model_and_task("runs/jupyter/run_0/config.yaml")

After the model has been trained, we can load it and visualize some of the planned trajectories.

In [ ]:
from usa.planners.clip_sdf import GradientPlanner

# Builds the planner from the model and task.
planner = GradientPlanner(
    dataset=task._dataset,
    model=model,
    task=task,
    device=task._device,

    # The learning rate for the optimizer for the waypoints
    lr=1e-2,
    # The weight for the total path distance loss term
    dist_loss_weight=1.0,
    # The weight for the inter-point distance loss term
    spacing_loss_weight=1.0,
    # The weight for the "no-crashing-into-a-wall" loss term
    occ_loss_weight=25.0,
    # The weight for the loss term of the final semantic location
    sim_loss_weight=15.0,
    # Maximum number of optimization steps
    num_optimization_steps=1000,
    # If points move less than this distance, stop optimizing
    min_distance=1e-5,
    # Where to store cache artifacts
    # cache_dir=Path("cache"),
    cache_dir=None,
    # Height of the floor
    floor_height=0.1,
    # Height of the ceiling
    ceil_height=2.0,
)

We plot the occupancy grid below.

In [ ]:
plt.imshow(planner.occ_map.grid)

In [ ]:
planner.occ_map.is_occupied(planner.occ_map.to_pt((-6.0, -1.0)))

In [ ]:
# waypoints = [(-5.0 + i * 0.1, 0.0) for i in range(20)]
waypoints_1 = planner.plan(start_xy=(-6.0, -2.0), end_goal="The oven")
x, y = waypoints_1[-1]
waypoints_2 = planner.plan(start_xy=(x, y), end_xy=(x, y + 1.0))
waypoints = waypoints_1 + waypoints_2

Now that we've got our trajectory, we can run it in simulation.

In [ ]:
def move_camera(
    camera_xyz: tuple[float, float, float],
    camera_ypr: tuple[float, float, float],
) -> tuple[tuple[float, float, float], tuple[float, float, float]]:
    x, y, z = camera_xyz
    yaw, pitch, roll = camera_ypr
    return (
        (x - 0.005, y + 0.01, z),
        (yaw + 0.1, pitch, roll),
    )


def capture_sim(total_steps: int = 100, capture_every: int = 1) -> Iterator[tuple[np.ndarray, ...]]:
    camera_xyz = (-5.0, 0.0, 1.477)
    camera_ypr = (90.0, -10.0, 0.0)
    for i in range(total_steps):
        camera_xyz, camera_ypr = move_camera(camera_xyz, camera_ypr)
        if i % capture_every == 0:
            yield capture_frame(camera_xyz, camera_ypr)


def iter_frames_for_waypoints(waypoints: list[tuple[float, float]], *, camera_z: float = 1.477, camera_ypr: tuple[float, float, float] = (90.0, -10.0, 0.0)) -> Iterator[np.ndarray]:
    pb.resetSimulation()
    kitchen = Kitchen()
    for x, y in waypoints:
        yield capture_frame((x + 3.0, y, camera_z), camera_ypr)


pkl_path = data_root / "04_recorded_clip_waypoints.pkl"
write_gif(iter_frames_for_waypoints(waypoints), "waypoints_video.gif", pkl_path)
Image("waypoints_video.gif")